In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import udf 
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import CountVectorizer, StringIndexer
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## Load the data

In [2]:
spark = SparkSession.builder.appName("SMS").getOrCreate()

In [3]:
data = spark.read.format('text').load('SMSSpamCollection.txt')
split_col = pyspark.sql.functions.split(data['value'], '\t')
data = data.withColumn('label', split_col.getItem(0))
data = data.withColumn('text', split_col.getItem(1)).drop('value')

In [4]:
data.show(n=5)

+-----+--------------------+
|label|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



## Tokenization

In [5]:
count_words = udf(lambda words: len(words), IntegerType())
tokenizer = RegexTokenizer(inputCol='text', outputCol='words', pattern='\\W')
regex_df = tokenizer.transform(data)
regex_tokenized_counts = regex_df.withColumn('freq', count_words('words'))
regex_tokenized_counts.show(n=5)

+-----+--------------------+--------------------+----+
|label|                text|               words|freq|
+-----+--------------------+--------------------+----+
|  ham|Go until jurong p...|[go, until, juron...|  20|
|  ham|Ok lar... Joking ...|[ok, lar, joking,...|   6|
| spam|Free entry in 2 a...|[free, entry, in,...|  33|
|  ham|U dun say so earl...|[u, dun, say, so,...|  11|
|  ham|Nah I don't think...|[nah, i, don, t, ...|  14|
+-----+--------------------+--------------------+----+
only showing top 5 rows



## Remove stopwords

In [6]:
remover = StopWordsRemover(inputCol='words', outputCol='tokens')
tokens_filtered = remover.transform(regex_tokenized_counts)
cleanDF= tokens_filtered.withColumn('count_tokens', count_words('tokens'))
cleanDF.show(n=5)

+-----+--------------------+--------------------+----+--------------------+------------+
|label|                text|               words|freq|              tokens|count_tokens|
+-----+--------------------+--------------------+----+--------------------+------------+
|  ham|Go until jurong p...|[go, until, juron...|  20|[go, jurong, poin...|          16|
|  ham|Ok lar... Joking ...|[ok, lar, joking,...|   6|[ok, lar, joking,...|           6|
| spam|Free entry in 2 a...|[free, entry, in,...|  33|[free, entry, 2, ...|          25|
|  ham|U dun say so earl...|[u, dun, say, so,...|  11|[u, dun, say, ear...|           9|
|  ham|Nah I don't think...|[nah, i, don, t, ...|  14|[nah, think, goes...|           7|
+-----+--------------------+--------------------+----+--------------------+------------+
only showing top 5 rows



## N-grams

In [7]:
ngram = NGram(n=2, inputCol='tokens', outputCol='2grams')
my_2ngrams =ngram.transform(cleanDF)
my_2ngrams.show(n=5)

+-----+--------------------+--------------------+----+--------------------+------------+--------------------+
|label|                text|               words|freq|              tokens|count_tokens|              2grams|
+-----+--------------------+--------------------+----+--------------------+------------+--------------------+
|  ham|Go until jurong p...|[go, until, juron...|  20|[go, jurong, poin...|          16|[go jurong, juron...|
|  ham|Ok lar... Joking ...|[ok, lar, joking,...|   6|[ok, lar, joking,...|           6|[ok lar, lar joki...|
| spam|Free entry in 2 a...|[free, entry, in,...|  33|[free, entry, 2, ...|          25|[free entry, entr...|
|  ham|U dun say so earl...|[u, dun, say, so,...|  11|[u, dun, say, ear...|           9|[u dun, dun say, ...|
|  ham|Nah I don't think...|[nah, i, don, t, ...|  14|[nah, think, goes...|           7|[nah think, think...|
+-----+--------------------+--------------------+----+--------------------+------------+--------------------+
only showi

## TF-IDF

In [8]:
tf = HashingTF(inputCol='tokens', outputCol='fs')
tf_df = tf.transform(my_2ngrams)
tf_df.select('fs').show(n=5)

+--------------------+
|                  fs|
+--------------------+
|(262144,[17222,26...|
|(262144,[122516,1...|
|(262144,[7958,122...|
|(262144,[28698,35...|
|(262144,[2710,259...|
+--------------------+
only showing top 5 rows



In [9]:
idf = IDF(inputCol='fs', outputCol='features')
idf_model = idf.fit(tf_df)
data = idf_model.transform(tf_df).select('label', 'features')
label_01 = udf(lambda lb: 1 if lb == 'ham' else 0, IntegerType()) #1 represents ham; 0 represents spam
data = data.withColumn('label', label_01('label'))
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(262144,[17222,26...|
|    1|(262144,[122516,1...|
|    0|(262144,[7958,122...|
|    1|(262144,[28698,35...|
|    1|(262144,[2710,259...|
|    0|(262144,[8443,240...|
|    1|(262144,[2089,497...|
|    1|(262144,[38868,53...|
|    0|(262144,[34651,43...|
|    0|(262144,[10951,40...|
|    1|(262144,[6258,178...|
|    0|(262144,[7958,139...|
|    0|(262144,[7958,209...|
|    1|(262144,[55639,71...|
|    1|(262144,[81948,13...|
|    0|(262144,[66,7958,...|
|    1|(262144,[18910,63...|
|    1|(262144,[739,3511...|
|    1|(262144,[30913,61...|
|    0|(262144,[7958,415...|
+-----+--------------------+
only showing top 20 rows



## Classification

In [10]:
train, test = data.randomSplit([0.7, 0.3])

### LogisticRegression

In [11]:
lr = LogisticRegression()
gridBuilder = ParamGridBuilder().addGrid(lr.maxIter, [50, 100, 200]).addGrid(lr.regParam, [0, 1, 2]).build()
evaluator = BinaryClassificationEvaluator()

In [12]:
cv = CrossValidator(estimator=lr, 
                    estimatorParamMaps=gridBuilder, 
                    evaluator=evaluator)

In [13]:
cvm = cv.fit(train)
evaluator.evaluate(cvm.transform(test))

0.9947683489350155

### Decision Tree

In [16]:
stringIndexer = StringIndexer(inputCol="label", outputCol="indexed")
si_model = stringIndexer.fit(train)
train_st = si_model.transform(train)
dt = DecisionTreeClassifier(labelCol='indexed')

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 1028, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/Nicolas/anaconda3/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 54] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: org does not exist in the JVM

In [15]:
cvm = dt.fit(train_st)

Py4JJavaError: An error occurred while calling o2978.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 967.0 failed 1 times, most recent failure: Lost task 0.0 in stage 967.0 (TID 939, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator$$anonfun$5.apply(ShuffleBlockFetcherIterator.scala:390)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator$$anonfun$5.apply(ShuffleBlockFetcherIterator.scala:390)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at org.apache.spark.util.Utils$$anonfun$copyStream$1.apply$mcJ$sp(Utils.scala:342)
	at org.apache.spark.util.Utils$$anonfun$copyStrea

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50226)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/Nicolas/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/Nicolas/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/Nicolas/anaconda3/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError
---------------

In [14]:
evaluator.evaluate(cvm.transform(test))

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 64109)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/Nicolas/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/Nicolas/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/Nicolas/anaconda3/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/serializers.py", line 577, in read_int
    raise EOFError
EOFError
---------------

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64094)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-24f7ad0102e4>", line 7, in <module>
    cvm = cv.fit(train)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 64, in fit
    return self._fit(dataset)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/tuning.py", line 232, in _fit
    models = est.fit(train, epm)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in fit
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in <listcomp>
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64094)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-24f7ad0102e4>", line 7, in <module>
    cvm = cv.fit(train)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 64, in fit
    return self._fit(dataset)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/tuning.py", line 232, in _fit
    models = est.fit(train, epm)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in fit
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in <listcomp>
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64094)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-24f7ad0102e4>", line 7, in <module>
    cvm = cv.fit(train)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 64, in fit
    return self._fit(dataset)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/tuning.py", line 232, in _fit
    models = est.fit(train, epm)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in fit
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in <listcomp>
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64094)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-24f7ad0102e4>", line 7, in <module>
    cvm = cv.fit(train)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 64, in fit
    return self._fit(dataset)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/tuning.py", line 232, in _fit
    models = est.fit(train, epm)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in fit
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in <listcomp>
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64094)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-24f7ad0102e4>", line 7, in <module>
    cvm = cv.fit(train)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 64, in fit
    return self._fit(dataset)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/tuning.py", line 232, in _fit
    models = est.fit(train, epm)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in fit
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in <listcomp>
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64094)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-24f7ad0102e4>", line 7, in <module>
    cvm = cv.fit(train)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 64, in fit
    return self._fit(dataset)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/tuning.py", line 232, in _fit
    models = est.fit(train, epm)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in fit
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in <listcomp>
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64094)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-24f7ad0102e4>", line 7, in <module>
    cvm = cv.fit(train)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 64, in fit
    return self._fit(dataset)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/tuning.py", line 232, in _fit
    models = est.fit(train, epm)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in fit
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in <listcomp>
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:64094)
Traceback (most recent call last):
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-24f7ad0102e4>", line 7, in <module>
    cvm = cv.fit(train)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 64, in fit
    return self._fit(dataset)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/tuning.py", line 232, in _fit
    models = est.fit(train, epm)
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in fit
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/Users/Nicolas/anaconda3/lib/python3.6/site-packages/pyspark/ml/base.py", line 59, in <listcomp>
    return [self.fit(dataset, paramMap) for paramMap in params]
  File "/

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:64094)